In [2]:
pip install confluent_kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 30.0 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip show confluent_kafka  #ensures package remains intalled incase of interuptn/restart

Name: confluent-kafka
Version: 2.11.0
Summary: Confluent's Python client for Apache Kafka
Home-page: https://github.com/confluentinc/confluent-kafka-python
Author: 
Author-email: "Confluent Inc." <support@confluent.io>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [ ]:
import json 
import os
import pandas as pd
from confluent_kafka import Producer

#----setting-up kafka-producer config

conf ={
    "bootstrap.servers":"pkc-p11xm.us-east-1.aws.confluent.cloud:9092",
    "security.protocol":"SASL_SSL",
    "sasl.mechanisms":"PLAIN",
    "sasl.username":"3O7M2WQA3FS7P6LG",
    "sasl.password":"ztYmJnQkOtfuB033bv/lJXjmD9MKeeqQUOpuniRpRm0RkkGguaRQUajj9oaYMl47",
    "client.id":"json-serial-producer"
}

producer=Producer(conf)   #instance of prod class
topic = "raw_topic"

def delivery_report(err,msg):
    if err:
        print(f"Message delivery failed:{err}")
    else:
        print(f"message delivered:{msg.key()}")

def read_checkpoint(checkpoint_file):
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file,'r') as file:
            return int(file.read().strip())
    return 0                                 #return 0 if file doesnt exists

def write_checkpoint(checkpoint_file,index):  
    with open(checkpoint_file,'w') as file:
        file.write(str(index))             #added curr-idx to keep track of sent msgs 
    print(f"checkpoint updated to line:{index}")

def handle_date(obj):
    if isinstance(obj,pd.Timestamp):
        return obj.strftime('%Y-%m-%d %H:%M:%S')  #returns by converting to string
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")


#MAIN FUNC

def stream_json_serially(file_path,checkpoint_file='/kaggle/working/checkpoint.txt'):
    last_sent_index=read_checkpoint(checkpoint_file)    #tells from where to resume

    with open(file_path,'r') as file:
        for idx,line in enumerate(file):
            if idx < last_sent_index:
                continue
            try:
                record = json.loads(line)
                producer.produce(
                    topic,
                    key=str(record['review_id']),
                    value=json.dumps(record,default=handle_date).encode('utf-8'),
                    callback=delivery_report
                )
                producer.flush()
                
                write_checkpoint(checkpoint_file,idx+1)
                
            except json.JSONDecodeError as e:
                print(f"Failed to decode JSON:{e}")

if __name__ == "__main__" :
    stream_json_serially('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json')
    
               
                
    

%6|1753464469.435|GETSUBSCRIPTIONS|json-serial-producer#producer-1| [thrd:main]: Telemetry client instance id changed from AAAAAAAAAAAAAAAAAAAAAA to /MxN7fO7SF+1HkEMo1c67g


message delivered:b'2grSsWx36eOA8CB8HrPh0g'
checkpoint updated to line:33193
message delivered:b'VUXNKY_3WD_TgY8GPa7y3w'
checkpoint updated to line:33194
message delivered:b'GLPNn3t6U5hpnqSNM5k-GA'
checkpoint updated to line:33195
message delivered:b'91Phv9wbb7qkhooUSqJzaA'
checkpoint updated to line:33196
message delivered:b'rFVF09yfsJE4WEPu9skLIw'
checkpoint updated to line:33197
message delivered:b'Et1GNktg5xQz3YO3518mZQ'
checkpoint updated to line:33198
message delivered:b'_7jwZ7-utqCrdjjgsLradQ'
checkpoint updated to line:33199
message delivered:b't5sBpj31UqDuysKeOMa11Q'
checkpoint updated to line:33200
message delivered:b'YNxKBV1760_nWyFKAri2Hg'
checkpoint updated to line:33201
message delivered:b'N42V3Bo-vFVDjGcKrdhdEQ'
checkpoint updated to line:33202
message delivered:b'McZejUoim0t8TMJ1g0OlZQ'
checkpoint updated to line:33203
message delivered:b'mhlDdLk_des3t5evmMBGCg'
checkpoint updated to line:33204
message delivered:b'YnhoOfXHMB2d7OqHGr9uRA'
checkpoint updated to line:33205